In [1]:
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

import pandas as pd
pd.options.display.max_columns = None
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#import mlflow

import torch
from torch.utils.data import Dataset, DataLoader

from transformers import AutoModel, AutoTokenizer

2023-05-03 11:50:07.656080: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 11:50:47.813701: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-03 11:50:47.813921: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-03 11:50:47.813930: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
print(device)

cuda:0


In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# file_name = "./dataset/cleaned_data.csv"
# file_name = "./dataset/preprocessed_data.csv"
file_name = "./dataset/merged_data.csv"

df = pd.read_csv(file_name)
df = df.rename({'Sentence' : 'text', 'Emotion' : 'label'}, axis=1)
LABEL = 'label'

# df > X, y
X, y = df.drop(LABEL, axis=1), df.loc[:, LABEL]

# encoding label
label_encoder = LabelEncoder()
label_encoder.fit(y)
df['label'] = y = label_encoder.transform(y) # range(7) > ['anger', 'disgust', 'fear', 'happiness', 'neutralism', 'sadness', 'surprise']
# df['label'] = y = label_encoder.transform(y) # range(7) > ['당황', '분노', '불안', '슬픔', '중립', '행복', '혐오']


n_classes = len(label_encoder.classes_)

# split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True, stratify=y)

trainset, testset = train_test_split(df, test_size=0.3, random_state=42, shuffle=True, stratify=df['label'])

In [4]:
label_encoder.inverse_transform(range(7))

array(['당황', '분노', '불안', '슬픔', '중립', '행복', '혐오'], dtype=object)

---

In [5]:
trainset['text'].apply(len).describe()

count    66000.000000
mean        70.652303
std         43.518038
min          1.000000
25%         26.000000
50%         75.000000
75%        107.000000
max        307.000000
Name: text, dtype: float64

In [6]:
testset['text'].apply(len).describe()

count    28287.000000
mean        70.904514
std         43.758114
min          1.000000
25%         26.000000
50%         76.000000
75%        107.000000
max        300.000000
Name: text, dtype: float64

In [7]:
# https://huggingface.co/klue/bert-base

# model = AutoModel.from_pretrained(CHECKPOINT)
# tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)
CHECKPOINT = "klue/bert-base"

In [8]:
class TokenDataset(Dataset):
    def __init__(self, dataframe, tokenizer_pretrained=CHECKPOINT, token_max_length=512):
        self.data = dataframe        
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_pretrained)
        self.token_max_len = token_max_length
  
    def __len__(self):
        return len(self.data)
  
    def __getitem__(self, idx):
        sentence = self.data.iloc[idx]['text']
        label = self.data.iloc[idx]['label']

        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                
            return_tensors='pt',     
            truncation=True,         
            padding='max_length',    
            add_special_tokens=True,
            max_length=self.token_max_len # max_len
        )

        input_ids = tokens['input_ids'].squeeze(0)           # 2D -> 1D
        attention_mask = tokens['attention_mask'].squeeze(0) # 2D -> 1D
        token_type_ids = torch.zeros_like(attention_mask)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask, 
            'token_type_ids': token_type_ids,
        }, torch.tensor(label)

In [9]:
model_params = dict(
    token_len = 64,
    batch_size = 16,
    drop_out = 0.5,
)

train_data = TokenDataset(trainset, CHECKPOINT, model_params['token_len'])
test_data = TokenDataset(testset, CHECKPOINT, model_params['token_len'])

train_loader = DataLoader(train_data, batch_size=model_params['batch_size'], shuffle=True, num_workers=8)
test_loader = DataLoader(test_data, batch_size=model_params['batch_size'], shuffle=True, num_workers=8)

In [10]:
class Bert(torch.nn.Module):
    def __init__(self, bert_pretrained=CHECKPOINT, dropout_rate=0.3):
        super(Bert, self).__init__()
        self.bert = AutoModel.from_pretrained(bert_pretrained)
        self.dr = torch.nn.Dropout(p=dropout_rate)
        self.fc = torch.nn.Linear(768, n_classes)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        last_hidden_state = output['last_hidden_state']
        x = self.dr(last_hidden_state[:, 0, :])
        x = self.fc(x)
        return x

In [11]:
bert = Bert(bert_pretrained=CHECKPOINT, dropout_rate=model_params['drop_out'])
bert.to(device)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Bert(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       

In [12]:
train_params = dict(
    lr = 2e-5,
    weight_decay = 1e-2,
)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(bert.parameters(), lr=train_params['lr'], weight_decay=train_params['weight_decay'])

In [13]:
from tqdm import tqdm 

def model_train(model, data_loader, loss_fn, optimizer, device):
    model.train()
    
    running_loss = 0
    corr = 0
    counts = 0
    
    prograss_bar = tqdm(data_loader, unit='batch', total=len(data_loader), mininterval=1)
    
    for idx, (inputs, labels) in enumerate(prograss_bar):
        inputs = {k:v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)
        
        optimizer.zero_grad()
        output = model(**inputs)
        loss = loss_fn(output, labels)
        
        loss.backward()
        optimizer.step()
        
        _, pred = output.max(dim=1)
        
        corr += pred.eq(labels).sum().item()
        counts += len(labels)
        
        running_loss += loss.item() * labels.size(0)
        
        prograss_bar.set_description(f"training loss: {running_loss/(idx+1):.5f}, training accuracy: {corr / counts:.5f}")
        
    acc = corr / len(data_loader.dataset)
    
    return running_loss / len(data_loader.dataset), acc

def model_evaluate(model, data_loader, loss_fn, device):
    model.eval()
    
    with torch.no_grad():
        corr = 0
        running_loss = 0
        
        for inputs, labels in data_loader:
            inputs = {k:v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)
            
            output = model(**inputs)
            
            _, pred = output.max(dim=1)
            
            corr += torch.sum(pred.eq(labels)).item()
            
            running_loss += loss_fn(output, labels).item() * labels.size(0)
        
        acc = corr / len(data_loader.dataset)
        
        return running_loss / len(data_loader.dataset), acc

In [14]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [51]:
# import mlflow

# mlflow.set_tracking_uri('mlruns')
# mlflow.set_experiment("Emotion_Classification")

In [17]:
# 에폭별 모델 저장하도록 코드 수정
if not os.path.exists("models"):
    os.mkdir("models")

model_base_path = "models/multi-dataset"
if not os.path.exists(model_base_path):
    os.mkdir(model_base_path)
    
model_name_base = 'klue-bert-' + "-".join([k+"_"+str(v) for k, v in model_params.items()] +[k+"_"+str(v) for k, v in train_params.items()])

num_epochs = 10
min_loss = np.inf
best_epoch = -1

for epoch in range(num_epochs):
    train_loss, train_acc = model_train(bert, train_loader, loss_fn, optimizer, device)
    val_loss, val_acc = model_evaluate(bert, test_loader, loss_fn, device)
    
    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        best_epoch = epoch
    
    torch.save(bert.state_dict(), f'{model_base_path + model_name_base} + {epoch}.pth')
    
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')

os.rename(f'{model_base_path + model_name_base} + {best_epoch}.pth', f'{model_base_path + model_name_base} + {best_epoch}_best.pth')
print(f'best val loss at {best_epoch}')

training loss: 15.66510, training accuracy: 0.62673: 100%|██████████| 4125/4125 [16:58<00:00,  4.05batch/s]


[INFO] val_loss has been improved from inf to 0.85483. Saving Model!
epoch 01, loss: 0.97907, acc: 0.62673, val_loss: 0.85483, val_accuracy: 0.67625


training loss: 12.14566, training accuracy: 0.71364: 100%|██████████| 4125/4125 [16:45<00:00,  4.10batch/s]


[INFO] val_loss has been improved from 0.85483 to 0.84886. Saving Model!
epoch 02, loss: 0.75910, acc: 0.71364, val_loss: 0.84886, val_accuracy: 0.67982


training loss: 9.49531, training accuracy: 0.77935: 100%|██████████| 4125/4125 [16:11<00:00,  4.25batch/s]


...

epoch 08, loss: 0.10802, acc: 0.96227, val_loss: 1.65987, val_accuracy: 0.67010


training loss: 1.44992, training accuracy: 0.96906: 100%|██████████| 4125/4125 [16:17<00:00,  4.22batch/s]


epoch 09, loss: 0.09062, acc: 0.96906, val_loss: 1.65875, val_accuracy: 0.67377


training loss: 1.26367, training accuracy: 0.97285: 100%|██████████| 4125/4125 [16:17<00:00,  4.22batch/s]


epoch 10, loss: 0.07898, acc: 0.97285, val_loss: 1.73524, val_accuracy: 0.67165
best val loss at 1


# Prediction

In [18]:
class EmotionClassifier():
    def __init__(self, model, tokenizer, labels: dict):
        model.to(device)
        self.model = model
        self.tokenizer = tokenizer
        self.labels = labels
        
    def predict(self, sentence):
        tokens = self.tokenizer(
            sentence,                # 1개 문장 
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True, # 스페셜 토큰 적용
            max_length = 64,
        )
        tokens.to(device)
        prediction = self.model(**tokens)
        prediction = torch.nn.functional.softmax(prediction, dim=1)
        output = prediction.argmax(dim=1).item()
        prob, result = prediction.max(dim=1)[0].item(), self.labels[output]
        print(f'[{result}]\n확률은: {prob*100:.3f}% 입니다.')

In [19]:
model = Bert()
model_path = '/home/wonhong/workspace/Emotion_Detection/DL/models/multi-datasetklue-bert-token_len_64-batch_size_16-drop_out_0.5-lr_2e-05-weight_decay_0.01 + 9.pth'
model.load_state_dict(torch.load(model_path))

# ['anger', 'disgust', 'fear', 'happiness', 'neutralism', 'sadness', 'surprise']
# ['당황', '분노', '불안', '슬픔', '중립', '행복', '혐오']
label_dict = {idx:label for idx, label in enumerate(label_encoder.inverse_transform(range(n_classes)))}

tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)

ecf = EmotionClassifier(model, tokenizer, label_dict)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
ecf.predict(df.sample(1)['text'].values[0])

[분노]
확률은: 99.866% 입니다.


In [21]:
for predict_times in range(10):
    tmp_df = df.sample(1)
    sentence = tmp_df['text'].values[0]
    ans = tmp_df['label'].values[0]
    print("SENTENCE : ", sentence)
    %time ecf.predict(sentence)
    print(ans)
    print("-"*30)

SENTENCE :  어찌 노인네들이 자기주장만 펼치는지 아무튼 그런 노인들이 너무 싫고 그런 상황이 슬퍼.노인정에서 어디를 놀러 가기로 했는데 서로 자기주장만 펼치잖아. 이 상황에 환멸이 나.서로 조금씩 양보하자고 건의해보려고. 그러면 조금씩 이해하는 마음이 생기지 않을까 해.
[슬픔]
확률은: 99.994% 입니다.
CPU times: user 68.3 ms, sys: 0 ns, total: 68.3 ms
Wall time: 68.4 ms
3
------------------------------
SENTENCE :  일을 안 다니니 창피해서 동창 모임에 안 나갈 거야.다들 잘나가고 결혼도 하고 직업도 좋은데 나만 백수야.응. 스스로 자신감이 없어.
[당황]
확률은: 99.978% 입니다.
CPU times: user 37.6 ms, sys: 0 ns, total: 37.6 ms
Wall time: 37.7 ms
0
------------------------------
SENTENCE :  길을 걸어가다가 넘어졌는데 아내가 너무 웃어서 부끄러웠어.주위에 사람이 엄청 많아서 그런지 시선들이 느껴져서 더 부끄러웠어.집에 가서 아내에게 한 마디 해야겠어.
[당황]
확률은: 99.988% 입니다.
CPU times: user 29 ms, sys: 9.41 ms, total: 38.4 ms
Wall time: 38.3 ms
0
------------------------------
SENTENCE :  드디어 회사 프로젝트가 끝나서 너무 기뻐.당분간은 야근이 없을 것 같아.못 만난 친구들과 수다도 떨고 맛있는 걸 먹으러 돌아다니고 싶어.
[행복]
확률은: 99.993% 입니다.
CPU times: user 28.2 ms, sys: 0 ns, total: 28.2 ms
Wall time: 28.3 ms
5
------------------------------
SENTENCE :  나 요즘 시험점수때문에 많이 힘들어.최근 내가 공부한것만큼 성과가 안 나오는것 같아.

---